# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
readpath = Path.cwd() / 'cities.csv'
df = pd.read_csv(readpath, index_col=0)
df.head()

,Long Guess,CityName,CountryCode,Temperature,Humidity,Cloudiness,Wind Speed,Lat Real,Long Real,Datetime
Lat Guess,,,,,,,,,,
-29.922669,154.099067,coffs harbour,au,69.80,73.0,90.0,10.29,-30.30,153.13,1.603847e+09
59.795476,93.428505,severo-yeniseyskiy,ru,21.90,97.0,100.0,12.19,60.37,93.04,1.603847e+09
-27.510159,174.227503,kaeo,nz,72.00,73.0,59.0,5.01,-35.10,173.78,1.603847e+09
80.980629,-163.606161,barrow,us,30.20,74.0,75.0,25.28,71.29,-156.79,1.603847e+09
-18.084154,-75.935205,acari,pe,66.45,67.0,79.0,3.98,-15.43,-74.62,1.603847e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [32]:
Humiditylocations = df[['Lat Real', 'Long Real']]
weights = df['Humidity']*10000
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(Humiditylocations, weights=weights, point_radius=20))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    - 65-85 F
    - Wind Speed <15
    - 50% Cloudiness
* Drop any rows will null values.

In [33]:
# Filter a COPY of the original dataframe based on above parameters for a "perfect day"
df_copy = df.copy()
df_copy = df_copy.loc[df_copy['Temperature'] >= 70 ]
df_copy = df_copy.loc[df_copy['Temperature'] <= 85 ]
df_copy = df_copy.loc[df_copy['Wind Speed'] <= 10]
df_copy = df_copy.loc[df_copy['Cloudiness'] <= 50]
print(f'The remaining number of cities that fit our criteria is {len(df_copy)}')
df_copy.to_csv(Path.cwd() / 'testing.csv')

The remaining number of cities that fit our criteria is 60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
# Set up our basic query
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "distance": 5000,
    "rankby": "distance",
    "key": g_key
}

In [35]:
df_copy.reset_index(inplace=True)
df_copy.head()

,Lat Guess,Long Guess,CityName,CountryCode,Temperature,Humidity,Cloudiness,Wind Speed,Lat Real,Long Real,Datetime
0,13.391774,-42.612194,cayenne,gf,82.40,78.0,20.0,3.36,4.93,-52.33,1.603847e+09
1,-17.231414,110.517043,karratha,au,80.20,57.0,18.0,9.95,-20.74,116.85,1.603847e+09
2,9.445800,-74.286498,pijino,co,79.30,80.0,10.0,2.68,9.33,-74.45,1.603847e+09
3,-10.132149,62.641552,victoria,sc,81.34,74.0,15.0,8.81,-4.62,55.45,1.603847e+09
4,-12.899780,9.985512,namibe,ao,71.11,79.0,0.0,3.27,-15.20,12.15,1.603847e+09


In [36]:
# Store into variable named hotel_df
hotel_df = df_copy
hotel_df['Hotel Name'] = ''
for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Lat Real']},{row['Long Real']}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest lodging is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan # Added this to make it possible to use .dropna()

Closest lodging is Hotel Central.
Closest lodging is Karratha International Hotel.
Closest lodging is Hotel Sara Inn.
Closest lodging is Bel Air Hotel.
Closest lodging is Hotel Chik Chik Namibe.
Closest lodging is Super Star Tourist Inn (Pvt) Ltd.
Closest lodging is Hostel Tigre.
Closest lodging is Hotel National.
Closest lodging is Ashwini delux lodge.
Closest lodging is Star Emirates Furnished Apartments.
Closest lodging is Travellers Haven.
Closest lodging is Auberge De Souillac.
Closest lodging is Bel Air Hotel.
Closest lodging is Hotel Chik Chik Namibe.
Closest lodging is Yako.
Missing field/result... skipping.
Closest lodging is Mandara Kauai.
Closest lodging is فندق المنشي دمياط.
Closest lodging is Hotel Chik Chik Namibe.
Closest lodging is Sikakubete Guest House Kalabo Town Council.
Closest lodging is Sharma Hotel and Restaurant.
Closest lodging is BACKHOUSE.
Closest lodging is Garden Lake Villa.
Closest lodging is Ko Olina Resort.
Closest lodging is Restaurante e Pousada Front

In [37]:
hotel_df.rename(columns={"CityName": "City", "CountryCode": "Country", "Lat Real": "Lat", "Long Real": "Lng"}, inplace=True)
hotel_df.dropna(inplace=True)

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))